In [23]:
import numpy as np
import networkx as nx
import json
import torch


c:\Users\Lenovo\anaconda3\envs\py39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
inj = np.load('injections.npy')
loads = np.load('loads.npy')

In [57]:
inj[:5]

array([[ 65.90000439,  51.19999695, -15.30000305, -45.79999924,
         -7.        ,  24.40000153,   0.        ,   0.        ,
        -28.20000076,  -8.69999981,  -3.5       ,  -5.5       ,
        -12.69999981, -14.80000019],
       [ 67.90000153,  51.20000267, -16.40000153, -45.90000153,
         -6.9000001 ,  24.29999828,   0.        ,   0.        ,
        -28.39999962,  -8.80000019,  -3.4000001 ,  -5.5       ,
        -12.89999962, -15.19999981],
       [ 66.00000286,  51.70000076, -15.40000153, -45.        ,
         -6.9000001 ,  23.39999866,   0.        ,   0.        ,
        -28.10000038,  -8.89999962,  -3.4000001 ,  -5.5       ,
        -12.80000019, -15.10000038],
       [ 65.99999285,  52.20000267, -15.69999695, -44.79999924,
         -6.9000001 ,  23.40000153,   0.        ,   0.        ,
        -28.60000038,  -8.89999962,  -3.4000001 ,  -5.4000001 ,
        -12.80000019, -15.10000038],
       [ 68.0999918 ,  51.80000305, -16.19999695, -45.29999924,
         -6.80000019

In [60]:
loads.shape

(10000, 20)

In [30]:
with open('adjacency.json') as f:
    branch_info = json.load(f)

# Initialize node features and edge features
nodes = inj
edges = np.zeros((len(branch_info), 1))  # Assuming 1-dimensional edge feature for simplicity

# Assign node features based on injection vectors
print("Node Features:")
print(nodes)

# Assign edge features based on branch reactance
for branch_id, info in branch_info.items():
    from_bus, to_bus, reactance = info["from"], info["to"], info["reactance"]
    edges[int(branch_id)] = reactance

print("\nEdge Features:")
print(edges)

Node Features:
[[ 65.90000439  51.19999695 -15.30000305 ...  -5.5        -12.69999981
  -14.80000019]
 [ 67.90000153  51.20000267 -16.40000153 ...  -5.5        -12.89999962
  -15.19999981]
 [ 66.00000286  51.70000076 -15.40000153 ...  -5.5        -12.80000019
  -15.10000038]
 ...
 [ 65.70000505  52.10000038  -3.30000305 ...  -5.4000001  -10.60000038
  -11.39999962]
 [ 65.60000467  52.79999924  -3.1000061  ...  -5.5        -10.5
  -11.30000019]
 [ 64.6999979   51.60000038  -2.5        ...  -5.4000001  -10.39999962
  -11.19999981]]

Edge Features:
[[0.05917]
 [0.22304]
 [0.19797]
 [0.17632]
 [0.17388]
 [0.17103]
 [0.04211]
 [0.1989 ]
 [0.25581]
 [0.13027]
 [0.0845 ]
 [0.27038]
 [0.19207]
 [0.19988]
 [0.34802]
 [0.20912]
 [0.55618]
 [0.25202]
 [0.17615]
 [0.11001]]


In [31]:
injection_vectors = inj
# Load branch information
with open('adjacency.json', 'r') as f:
    branch_info = json.load(f)

# Create a graph using networkx
graph = nx.Graph()

# Add nodes with injection vectors as features
for i in range(len(injection_vectors)):
    node_features = injection_vectors[i, :]
    graph.add_node(i, features=torch.tensor(node_features, dtype=torch.float32))

# Add edges with reactance as features
for key, value in branch_info.items():
    from_node = value['from']
    to_node = value['to']
    reactance = value['reactance']
    graph.add_edge(from_node, to_node, features=torch.tensor(reactance, dtype=torch.float32))

# Access node and edge features
node_features = [graph.nodes[i]['features'] for i in graph.nodes]
edge_features = [graph.edges[i, j]['features'] for i, j in graph.edges]

# Convert node and edge features to tensors
node_features_tensor = torch.stack(node_features)
edge_features_tensor = torch.stack(edge_features)

# Print the tensors
print("Node Features Tensor:")
print(node_features_tensor)

print("\nEdge Features Tensor:")
print(edge_features_tensor)

Node Features Tensor:
tensor([[ 65.9000,  51.2000, -15.3000,  ...,  -5.5000, -12.7000, -14.8000],
        [ 67.9000,  51.2000, -16.4000,  ...,  -5.5000, -12.9000, -15.2000],
        [ 66.0000,  51.7000, -15.4000,  ...,  -5.5000, -12.8000, -15.1000],
        ...,
        [ 65.7000,  52.1000,  -3.3000,  ...,  -5.4000, -10.6000, -11.4000],
        [ 65.6000,  52.8000,  -3.1000,  ...,  -5.5000, -10.5000, -11.3000],
        [ 64.7000,  51.6000,  -2.5000,  ...,  -5.4000, -10.4000, -11.2000]])

Edge Features Tensor:
tensor([0.0592, 0.2230, 0.1980, 0.1763, 0.1739, 0.1710, 0.0421, 0.2091, 0.5562,
        0.2520, 0.1989, 0.2558, 0.1303, 0.1761, 0.1100, 0.0845, 0.2704, 0.1921,
        0.1999, 0.3480])


In [33]:
nodes[0]

array([ 65.90000439,  51.19999695, -15.30000305, -45.79999924,
        -7.        ,  24.40000153,   0.        ,   0.        ,
       -28.20000076,  -8.69999981,  -3.5       ,  -5.5       ,
       -12.69999981, -14.80000019])

In [43]:
node_features_tensor.shape

torch.Size([10000, 14])

In [44]:
edge_features_tensor.shape

torch.Size([20])

In [46]:
import matplotlib.pyplot as plt

# Construct the graph
graph = nx.Graph()

# Add nodes with features
for i in range(node_features_tensor.shape[0]):
    graph.add_node(i, features=node_features_tensor[i].numpy())

# Add edges with features
for i, j in graph.edges:
    graph.add_edge(i, j, features=edge_features_tensor[i, j].numpy())

# # Visualize the graph (optional)
# pos = nx.spring_layout(graph)
# nx.draw(graph, pos, with_labels=True, font_weight='bold')
# plt.show()

In [54]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GraphConvolutionLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GraphConvolutionLayer, self).__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x, adj_matrix):
        # x: Node features, adj_matrix: Adjacency matrix
        x = self.linear(x)
        
        # Mean Aggregation
        degree_inv_sqrt = 1.0 / torch.sqrt(torch.sum(adj_matrix, dim=1))
        degree_inv_sqrt[torch.isinf(degree_inv_sqrt)] = 0.0  # Handle nodes with no neighbors
        normalized_adj_matrix = adj_matrix * degree_inv_sqrt[:, None]
        x = torch.matmul(normalized_adj_matrix, x)
        
        return x

class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.gc1 = GraphConvolutionLayer(input_dim, hidden_dim)
        self.gc2 = GraphConvolutionLayer(hidden_dim, output_dim)

    def forward(self, x, adj_matrix):
        x = F.relu(self.gc1(x, adj_matrix))
        x = self.gc2(x, adj_matrix)
        return x

# Assuming you have the adjacency matrix from the graph
# Construct the adjacency matrix
adj_matrix = nx.adjacency_matrix(graph).toarray()
adj_matrix = torch.tensor(adj_matrix, dtype=torch.float32)

# Assuming you have node features tensor
# Convert node features to tensor
node_features_tensor = torch.tensor(node_features_tensor, dtype=torch.float32)

# Create a GCN model with mean aggregation
input_dim = node_features_tensor.shape[1]
hidden_dim = 64  # Adjust as needed
output_dim = 20   # Output dimension for branch load prediction
gcn_model = GCN(input_dim, hidden_dim, output_dim)

# Forward pass
output = gcn_model(node_features_tensor, adj_matrix)

# Print the output shape
print("GCN Output Shape:", output.shape)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_16184\3253719275.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  node_features_tensor = torch.tensor(node_features_tensor, dtype=torch.float32)


GCN Output Shape: torch.Size([10000, 20])


In [55]:
import torch.optim as optim

# Assuming you have the target branch loads (ground truth)
# Replace this with your actual target tensor
target_branch_loads = torch.tensor(loads, dtype= torch.float32)

# Define the Mean Squared Error loss
criterion = nn.MSELoss()

# Define the Adam optimizer
optimizer = optim.Adam(gcn_model.parameters(), lr=0.001)

# Number of training epochs
epochs = 100

# Training loop
for epoch in range(epochs):
    # Forward pass
    output = gcn_model(node_features_tensor, adj_matrix)
    
    # Calculate the loss
    loss = criterion(output, target_branch_loads)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the loss for every few epochs
    if epoch % 10 == 0:
        print(f'Epoch [{epoch}/{epochs}], Loss: {loss.item()}')

# After training, you can use the trained model to make predictions
# For example, you can use gcn_model(node_features_tensor, adj_matrix) to get predictions


Epoch [0/100], Loss: 273.3698425292969
Epoch [10/100], Loss: 273.3698425292969
Epoch [20/100], Loss: 273.3698425292969
Epoch [30/100], Loss: 273.3698425292969
Epoch [40/100], Loss: 273.3698425292969
Epoch [50/100], Loss: 273.3698425292969
Epoch [60/100], Loss: 273.3698425292969
Epoch [70/100], Loss: 273.3698425292969
Epoch [80/100], Loss: 273.3698425292969
Epoch [90/100], Loss: 273.3698425292969


In [50]:
# import torch.optim as optim
# from sklearn.model_selection import RandomizedSearchCV

# # Define the MSE loss
# criterion = nn.MSELoss()

# # Define the hyperparameter search space
# param_dist = {
#     'hidden_dim': [32, 64, 128],
#     'output_dim': [1],  # Output dimension for branch load prediction
#     'lr': [0.001, 0.01, 0.1],
# }

# # Assuming you have target branch loads (labels)
# # Replace this with your actual target data
# target_branch_loads = torch.tensor(loads, dtype=torch.float32)

# # Define the training function
# def train_gcn(hidden_dim, output_dim, lr, input_dim, adj_matrix, node_features_tensor, target_branch_loads):
#     gcn_model = GCN(input_dim, hidden_dim, output_dim)
#     optimizer = optim.Adam(gcn_model.parameters(), lr=lr)

#     for epoch in range(100):  # Adjust the number of epochs as needed
#         optimizer.zero_grad()
#         output = gcn_model(node_features_tensor, adj_matrix)
#         loss = criterion(output.squeeze(), target_branch_loads)
#         loss.backward()
#         optimizer.step()

#     return loss.item()

# # Create the GCN model
# input_dim = node_features_tensor.shape[1]

# # Initialize RandomizedSearchCV
# random_search = RandomizedSearchCV(
#     train_gcn,
#     param_distributions=param_dist,
#     n_iter=5,  # Number of random parameter combinations to try
#     cv=3,  # Number of cross-validation folds
#     verbose=1,
#     random_state=42,
# )

# # Perform random search
# random_search.fit(
#     X_train=None,  # Replace with your training data if available
#     y_train=None,  # Replace with your training labels if available
#     adj_matrix=adj_matrix,
#     node_features_tensor=node_features_tensor,
#     target_branch_loads=target_branch_loads,
# )

# # Get the best hyperparameters
# best_params = random_search.best_params_

# # Print the best hyperparameters
# print("Best Hyperparameters:", best_params)
